In [50]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal

In [51]:
params = {
    # exp
    "exp_name": "real",
    "cv_folds": 5,
    "plot": True,
    "calib_methods": ["RF", "Platt", "ISO", "Rank", "CRF", "VA", "Beta", "Elkan", "tlr", "Line", 
                      "RF_boot", "RF_ens_r", "RF_ens_line", "RF_ens_CRF", "RF_ens_Platt", 
                      "RF_large", "RF_large_line"],
    "metrics": ["acc", "logloss", "brier", "ece", "auc"],
    
    # calib param
    "ece_bins": 40,
    "boot_size": 10,
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":10,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": [2,3,4,5,6,7,8,10,20,50,100],
                    "criterion": ["gini", "entropy"],
                    # "min_samples_split": [2,3,4,5],
                    # "min_samples_leaf": [1,2,3],
                    },
}

exp_key = "data_name"
exp_values = ["spambase", 
              "climate", 
              "QSAR", 
              "bank", 
              "parkinsons", 
              "vertebral", 
              "ionosphere", 
              "diabetes", 
              "breast", 
              "blod", 
              "hillvalley",
              "madelon",
              "scene",
              "spect",
              "Sonar_Mine_Rock_Data",
              "Customer_Churn"]

exp_values = ["ionosphere"]

In [52]:
calib_results_dict, data_list = cx.run_exp(exp_key, exp_values, params)
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)

ValueError: y_prob contains values greater than 1.

In [ ]:
tables["brier"]

,RF,Platt,ISO,Rank,CRF,VA,Beta,Elkan,tlr,Line,RF_boot,RF_ens_r,RF_ens_line,RF_ens_CRF,RF_ens_Platt,RF_large,RF_large_line
Data,,,,,,,,,,,,,,,,,
vertebral,0.172092,0.162355,0.154232,0.322581,0.172864,0.142288,0.148483,0.136142,0.290511,0.155209,0.171055,0.158326,0.124442,0.159941,0.115552,0.160363,0.121965
Mean,0.172092,0.162355,0.154232,0.322581,0.172864,0.142288,0.148483,0.136142,0.290511,0.155209,0.171055,0.158326,0.124442,0.159941,0.115552,0.160363,0.121965
Rank,14.000000,12.000000,7.000000,17.000000,15.000000,5.000000,6.000000,4.000000,16.000000,8.000000,13.000000,9.000000,3.000000,10.000000,1.000000,11.000000,2.000000


In [ ]:
tables["logloss"]

,RF,Platt,ISO,Rank,CRF,VA,Beta,Elkan,tlr,Line,RF_boot,RF_ens_r,RF_ens_line,RF_ens_CRF,RF_ens_Platt,RF_large,RF_large_line
Data,,,,,,,,,,,,,,,,,
vertebral,0.488814,0.479927,2.255278,11.626985,0.489885,0.429514,0.450944,0.424997,2.551283,0.449592,0.486817,0.44815,0.368064,0.448117,0.363322,0.453416,0.355195
Mean,0.488814,0.479927,2.255278,11.626985,0.489885,0.429514,0.450944,0.424997,2.551283,0.449592,0.486817,0.44815,0.368064,0.448117,0.363322,0.453416,0.355195
Rank,13.000000,11.000000,15.000000,17.000000,14.000000,5.000000,9.000000,4.000000,16.000000,8.000000,12.000000,7.00000,3.000000,6.000000,2.000000,10.000000,1.000000


In [ ]:
tables["acc"]

,RF,Platt,ISO,Rank,CRF,VA,Beta,Elkan,tlr,Line,RF_boot,RF_ens_r,RF_ens_line,RF_ens_CRF,RF_ens_Platt,RF_large,RF_large_line
Data,,,,,,,,,,,,,,,,,
vertebral,0.677419,0.741935,0.745161,0.677419,0.677419,0.790323,0.777419,0.819355,0.677419,0.745161,0.680645,0.677419,0.825806,0.677419,0.832258,0.677419,0.835484
Mean,0.677419,0.741935,0.745161,0.677419,0.677419,0.790323,0.777419,0.819355,0.677419,0.745161,0.680645,0.677419,0.825806,0.677419,0.832258,0.677419,0.835484
Rank,14.000000,9.000000,7.500000,14.000000,14.000000,5.000000,6.000000,4.000000,14.000000,7.500000,10.000000,14.000000,3.000000,14.000000,2.000000,14.000000,1.000000


In [ ]:
tables["ece"]

,RF,Platt,ISO,Rank,CRF,VA,Beta,Elkan,tlr,Line,RF_boot,RF_ens_r,RF_ens_line,RF_ens_CRF,RF_ens_Platt,RF_large,RF_large_line
Data,,,,,,,,,,,,,,,,,
vertebral,0.279054,0.257013,0.199233,0.0,0.277018,0.251695,0.246935,0.271004,0.261548,0.263472,0.278504,0.278258,0.248519,0.27529,0.204001,0.280386,0.246398
Mean,0.279054,0.257013,0.199233,0.0,0.277018,0.251695,0.246935,0.271004,0.261548,0.263472,0.278504,0.278258,0.248519,0.27529,0.204001,0.280386,0.246398
Rank,16.000000,8.000000,2.000000,1.0,13.000000,7.000000,5.000000,11.000000,9.000000,10.000000,15.000000,14.000000,6.000000,12.00000,3.000000,17.000000,4.000000
